In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# %pip install llama-index-readers-file
# %pip install llama-index-embeddings-huggingface
# %pip install llama-index-llms-huggingface
# %pip install llama-index-llms-llama-cpp

In [ ]:
# !pip install "transformers[torch]" "huggingface_hub[inference]"

In [ ]:
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python  --no-cache-dir

In [8]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [3]:
embeded_text = embed_model.get_text_embedding("hello Mapsa Iran Tehran!")
len(embeded_text)

384

In [16]:
import os

HF_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

In [17]:
from llama_index.llms.huggingface import (
    HuggingFaceInferenceAPI,
    HuggingFaceLLM,
)

# locally_run = HuggingFaceLLM(model_name="HuggingFaceH4/zephyr-7b-beta")

remotely_run = HuggingFaceInferenceAPI(
    model_name="HuggingFaceH4/zephyr-7b-beta", token=HF_TOKEN
)

/var/folders/t7/rf1zzxt969xddtxvl6f_hcsm0000gn/T/ipykernel_43006/850126269.py:8: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPI. (Deprecated in favor of `HuggingFaceInferenceAPI` from `llama-index-llms-huggingface-api` which should be used instead.)
  remotely_run = HuggingFaceInferenceAPI(


In [20]:
remotely_run.complete("what would be best public transportation solution between tramway, bus and subway?")

CompletionResponse(text="\n\nI'm planning to visit some places in Paris and I'm not familiar with the public transportation system. I'm wondering which one would be the most convenient and efficient way to travel between these places.\n\nHere are the places I want to visit:\n\n1. Eiffel Tower\n2. Louvre Museum\n3. Notre Dame Cathedral\n4. Montmartre\n5. Sacré-Cœur Basilica\n6. Champs-Élysées\n7. Arc de Triomphe\n8. Opera Garnier\n9. Saint-Germain-des-Prés\n10. Luxembourg Gardens\n\nI'm looking for a transportation option that is easy to use, frequent, and covers most of these places. Which one would you recommend?\n\nAlso, do you have any tips on how to navigate the public transportation system in Paris? I'm a bit overwhelmed by the number of lines and connections.", additional_kwargs={}, raw=None, logprobs=None, delta=None)

In [11]:
!pip show llama-index

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
from llama_index.core import SimpleDirectoryReader, Document

documents = SimpleDirectoryReader(
    "./data/"
).load_data()


In [10]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter


# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

# base node parser is a sentence splitter
text_splitter = SentenceSplitter()

from llama_index.core import Settings

Settings.llm = remotely_run
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

In [14]:
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

nodes = node_parser.get_nodes_from_documents(documents)  #len: 10290
base_nodes = text_splitter.get_nodes_from_documents(documents) #len: 743

Settings.llm = remotely_run
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

from llama_index.core import VectorStoreIndex

sentence_index = VectorStoreIndex(nodes)
# base_index = VectorStoreIndex(base_nodes)

from llama_index.core.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)
window_response = query_engine.query(
    "what would be best public transportation solution between tramway, bus and subway? "
)

print(window_response)



The best public transportation solution would depend on the specific location and the needs of the individual. However, based on the provided context, the tramway seems to have some advantages over bus and subway. 

Firstly, the tramway is expected to carry 10 million passengers per year, which is significantly higher than the current bus usage of 6.5 million passengers per year. This suggests that the tramway may provide a more efficient and reliable service.

Secondly, some 35% of the increased passenger traffic is expected to transfer from cars, which could help to reduce congestion and air pollution in the area. This is in contrast to the subway, which may not have the same level of transfer potential.

Thirdly, the tramway is expected to be fully accessible to the mobility impaired, which is an important consideration for many people. This is not necessarily the case for all bus services, and the subway may also have some accessibility issues.

Fourthly, the tramway fares are ex